In [ ]:
import matplotlib.pyplot as plt
import ipyplot
import numpy as np
import cv2
import math
import statistics
from scipy.signal import find_peaks
from collections import Counter

In [ ]:
def imshow(img):
    plt.imshow(img[0:1680, 0:1700])

In [ ]:
img_orig = cv2.imread("jigsawsqr.png")
img_orig = img_orig[70:420, 75:440]
h, w = img_orig.shape[:2]
img_orig = cv2.resize(img_orig, (4*w, 4*h))
imshow(img_orig);

In [ ]:
img_gray = cv2.cvtColor(img_orig, cv2.COLOR_BGR2GRAY)
imshow(img_gray);

In [ ]:
img_edges = cv2.Canny(image=img_gray, threshold1=100, threshold2=200)
imshow(img_edges);

In [ ]:
_, img_binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
raw_contours, _ = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
img = img_orig.copy()
cv2.drawContours(img, raw_contours, -1, (0, 255, 0), 2)
print("Number of detected contours: ", len(raw_contours))
imshow(img);

In [ ]:
class Item():
    def __init__(self, **kwargs):
        self.update(**kwargs)

    def update(self, **kwargs):
        self.__dict__.update(kwargs)

class LoopingList(list):
    def __getitem__(self, i):
        if isinstance(i, int):
            return super().__getitem__(i % len(self))
        else:
            return super().__getitem__(i)

In [ ]:
img_orig = cv2.imread("jigsawsqr.png")
h, w = img_orig.shape[:2]
img_orig = cv2.resize(img_orig, (4*w, 4*h))
imshow(img_orig);

In [ ]:
img_gray = cv2.cvtColor(img_orig, cv2.COLOR_BGR2GRAY)
imshow(img_gray);

In [ ]:
_, img_binary = cv2.threshold(img_gray, 127, 255, cv2.THRESH_BINARY)
contours = cv2.findContours(img_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[0]
img = img_orig.copy()
cv2.drawContours(img, contours, -1, (0, 255, 0), 4)
print("Number of detected contours: ", len(contours))
imshow(img);

In [ ]:
regions = [Item(contour=contour, area=cv2.contourArea(contour), rect=cv2.boundingRect(contour)) for contour in contours]
median_area = statistics.median([region.area for region in regions])
min_area = 0.5 * median_area
max_area = 2 * median_area
print(f"Ignore too big or small regions, median_area: {median_area}")

regions = [region for region in regions if 0.5 < region.area / median_area < 2]
print(f"Number of remaining regions: {len(regions)}")

img = img_orig.copy()
for region in regions:
    x, y, w, h = region.rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 3)

imshow(img);

In [ ]:
imgs = []
pieces = []
for region in regions:
    x, y, w, h = region.rect
    p = np.array([x, y])
    piece = Item(
        contour=region.contour - p,
        area=region.area,
        img_orig=img_orig[y:y+h, x:x+w],
        img_gray=img_gray[y:y+h, x:x+w],
        size=np.array([h, w])
    )
    pieces.append(piece)

    img = piece.img_orig.copy()
    cv2.drawContours(img, piece.contour, -1, (0, 255, 0), 3)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=75);

In [ ]:
imgs = []
for piece in pieces:
    min_area_rect = cv2.minAreaRect(piece.contour)
    (cx, cy), (sx, sy), angle_degrees = min_area_rect
    if sy < sx:
        angle_degrees = (angle_degrees + 90) % 360
        sx, sy = sy, sx
    min_area_rect = ((cx, cy), (sx, sy), angle_degrees)
    piece.update(
        min_area_rect=min_area_rect,
        angle_degrees=angle_degrees
    )

    img = piece.img_orig.copy()
    box = np.int0(cv2.boxPoints(min_area_rect))
    cv2.drawContours(img, [box], 0, (0, 255, 0), 3)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=75);

In [ ]:
for piece in pieces:
    h, w = piece.size
    img = cv2.linearPolar(piece.img_gray, (w/2, h/2), max(h, w), cv2.WARP_FILL_OUTLIERS)
    y0 = int(piece.angle_degrees * h / 360) % 360
    img = np.concatenate([img[y0:, :], img[:y0, :]])
    _, img_binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    piece.update(img_polar=img_binary)

ipyplot.plot_images([piece.img_polar for piece in pieces], img_width=75); 

In [ ]:
imgs = []

for piece in pieces:
    img_reversed = np.flip(piece.img_polar, axis=1)
    h, w = piece.size
    radius = w - np.argmax(img_reversed, axis=1)
    peak_indices = find_peaks(radius, prominence=5)[0]

    polar_peaks = [(radius[idx] * max(h, w) / w, idx * 360 / h) for idx in peak_indices]
    piece.update(polar_peaks=polar_peaks)   # list of [(peak_radius, peak_degrees)]

    img = piece.img_polar.copy()
    for idx in peak_indices:
        cv2.circle(img, (radius[idx], idx), 15, 180, 2)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=150);

In [ ]:
imgs = []

for piece in pieces:
    h, w = piece.size

    img = piece.img_orig.copy()
    for peak_radius, peak_degrees in piece.polar_peaks:
        peak_radians = math.radians(peak_degrees + piece.angle_degrees)
        x = w/2 + peak_radius * math.cos(peak_radians)
        y = h/2 + peak_radius * math.sin(peak_radians)
        cv2.circle(img, (int(x), int(y)), 15, (0, 200, 0), 2)

    peaks = {}  # key=quarter, value=[(abs(angle), angle, distance)] where angles are relative to the piece_angle
    for peak_radius, peak_degrees in piece.polar_peaks:
        quarter = peak_degrees // 90
        diff_degrees = peak_degrees % 180
        if diff_degrees > 90:
            diff_degrees -= 180
        peaks.setdefault(quarter, []).append((abs(diff_degrees), peak_radius, peak_degrees + piece.angle_degrees))

    corners = LoopingList()
    for quarter in range(4):
        _, peak_radius, peak_degrees = min(peaks.get(3 - quarter, [(0, 0, 0)]))
        peak_radians = math.radians(peak_degrees)
        x = w/2 + peak_radius * math.cos(peak_radians)
        y = h/2 + peak_radius * math.sin(peak_radians)
        corner = Item(point=np.array([int(x), int(y)]))
        corners.append(corner)

    for idx, corner in enumerate(corners):
        corner.update(prev=corners[idx-1], next=corners[idx+1])
    piece.update(corners=corners)
    
    cv2.line(img, piece.corners[0].point, piece.corners[2].point, (255, 0, 0), 3)
    cv2.line(img, piece.corners[1].point, piece.corners[3].point, (255, 0, 0), 3)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=75);

In [ ]:
imgs = []

def length(v):
    return v[0]**2 + v[1]**2

def sub_contour(c, idx0, idx1):
    if idx1 > idx0:
        return c[idx0:idx1]
    else:
        return np.concatenate([c[idx0:], c[:idx1]])

for piece in pieces:
    idx_list = LoopingList()
    for corner in piece.corners:
        distances = [length(p[0] - corner.point) for p in piece.contour]
        idx = distances.index(min(distances))
        idx_list.append(idx)
    
    edges = LoopingList()
    for i in range(4):
        contour = sub_contour(piece.contour, idx_list[i-1], idx_list[i])
        edge = Item(
            contour=contour,
            p0=piece.corners[i-1].point,
            p1=piece.corners[i].point,
            corner0=piece.corners[i-1],
            corner1=piece.corners[i]
        )
        edges.append(edge)
    piece.update(edges=edges)

    img = piece.img_orig.copy()
    cv2.drawContours(img, piece.edges[0].contour, -1, (0, 255, 0), 6)
    cv2.drawContours(img, piece.edges[1].contour, -1, (255, 0, 0), 6)
    cv2.drawContours(img, piece.edges[2].contour, -1, (0, 255, 0), 6)
    cv2.drawContours(img, piece.edges[3].contour, -1, (255, 0, 0), 6)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=75);

In [ ]:
imgs = []

for piece in pieces:
    nb_flats = 0
    img = piece.img_orig.copy()
    for idx, edge in enumerate(piece.edges):
        dx, dy = edge.p1 - edge.p0
        angle_radians = math.atan2(dy, dx)
        sin = math.sin(angle_radians)
        cos = math.cos(angle_radians)
        matrix = np.array([[cos, -sin], [sin, cos]])
        normalized_contour = (edge.contour - edge.p0) @ matrix  # first point at (0, 0), last point at (X, 0)
        heights = normalized_contour[:,0,1]
        min_height = min(heights)
        max_height = max(heights)
        if abs(max_height) + abs(min_height) < 10:
            edge_type = 0
            nb_flats += 1
            color = (0, 255, 0)
        elif abs(max_height) > abs(min_height):
            edge_type = 1
            color = (255, 0, 0)
        else:
            edge_type = -1
            color = (0, 0, 255)
        edge.update(
            arc_length=cv2.arcLength(edge.contour, closed=False),
            straight_length=math.sqrt(dx**2 + dy**2),
            height=max(abs(min_height), abs(max_height)),
            angle_degrees=math.degrees(angle_radians),
            type=edge_type,
            prev=piece.edges[idx-1],
            next=piece.edges[idx+1]
        )
        edge.update(
            normalized_prev_point=(edge.prev.p0 - edge.p0) @ matrix,
            normalized_next_point=(edge.next.p1 - edge.p1) @ matrix,
        )
        cv2.drawContours(img, edge.contour, -1, color, 6)

    piece.update(nb_flats=nb_flats)
    imgs.append(img)

ipyplot.plot_images(imgs, img_width=75);

In [ ]:
ipyplot.plot_class_tabs(images=[piece.img_orig for piece in pieces], labels=[piece.nb_flats for piece in pieces], img_width=50);

In [ ]:
nb_flats = Counter([piece.nb_flats for piece in pieces])

assert nb_flats[2] == 4
# H**2 - H*B/2 + I = 0
a = 1
b = - nb_flats[1] / 2
c = nb_flats[0]
delta = b**2 - 4*a*c
inner_height = int((-b - math.sqrt(delta)) / (2*a))
inner_width = int((-b + math.sqrt(delta)) / (2*a))
print(f"Size of puzzle: {2 + inner_width}x{2 + inner_height}")
assert inner_height * inner_width == nb_flats[0]
assert 2 * (inner_height + inner_width) == nb_flats[1]

In [ ]:
imgs = []
PAD = 50

for idx, piece in enumerate(pieces):
    h, w = piece.size
    for edge in piece.edges:
        img = piece.img_orig.copy()
        # cv2.drawContours(img, edge.contour, -1, (0, 255, 0), 5)
        img = cv2.copyMakeBorder(img, PAD, PAD, PAD, PAD, cv2.BORDER_CONSTANT)
        matrix = cv2.getRotationMatrix2D((PAD + w/2, PAD + h/2), edge.angle_degrees, 1)
        cv2.warpAffine(img, matrix, (w+2*PAD, h+2*PAD), img)
        contour = cv2.transform(edge.contour + PAD, matrix)
        p0 = contour[0][0]
        p1 = contour[-1][0]
        p_prev = p0 + edge.normalized_prev_point
        p_next = p1 + edge.normalized_next_point
        cv2.circle(img, (int(p0[0]), int(p0[1])), 15, (0, 255, 0), 3)
        cv2.circle(img, (int(p_prev[0]), int(p_prev[1])), 25, (0, 255, 0), 3)
        cv2.circle(img, (int(p1[0]), int(p1[1])), 15, (255, 0, 0), 3)
        cv2.circle(img, (int(p_next[0]), int(p_next[1])), 25, (255, 0, 0), 3)
        cv2.line(img, (0, int(p0[1])), (w + 2*PAD, int(p0[1])), (0, 255, 0), 2)
        imgs.append(img)

ipyplot.plot_images(imgs, img_width=75);

In [ ]:
all_edges = [edge for piece in pieces for edge in piece.edges]
all_features = [(
    edge.type,
    edge.prev.type,
    edge.next.type,
    edge.straight_length,
    edge.arc_length,
    edge.height,
    edge.normalized_prev_point[0],
    edge.normalized_prev_point[1],
    edge.normalized_next_point[0],
    edge.normalized_next_point[1]
    ) for edge in all_edges
]
male_features = np.array([feature for feature in all_features if feature[0] == 1])
female_features = np.array([feature for feature in all_features if feature[0] == -1])

index_x = 3
index_y = 4
index_c = 5
f = male_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker="v")
f = female_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker="^")

In [ ]:
male_after_flat_features = np.array([feature for feature in all_features if feature[0] == 1 and feature[1] == 0])
female_after_flat_features = np.array([feature for feature in all_features if feature[0] == -1 and feature[1] == 0])
male_before_flat_features = np.array([feature for feature in all_features if feature[0] == 1 and feature[2] == 0])
female_before_flat_features = np.array([feature for feature in all_features if feature[0] == -1 and feature[2] == 0])

index_x = 4
index_y = 5
index_c = 3
f = male_before_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker=".")
f = female_after_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker="*")
f = male_after_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker="v")
f = female_before_flat_features; plt.scatter(x=f[:, index_x], y=f[:, index_y], c=f[:, index_c], marker="^");

In [ ]:
flat_after_male_features = np.array([feature for feature in all_features if feature[0] == 0 and feature[1] == 1])
flat_after_female_features = np.array([feature for feature in all_features if feature[0] == 0 and feature[1] == -1])
flat_before_male_features = np.array([feature for feature in all_features if feature[0] == 0 and feature[2] == 1])
flat_before_female_features = np.array([feature for feature in all_features if feature[0] == 0 and feature[2] == -1])

index_c = 3
f = flat_after_male_features; plt.scatter(x=f[:, 6], y=f[:, 7], c=f[:, index_c], marker=".")
f = flat_before_female_features; plt.scatter(x=f[:, 8], y=f[:, 9], c=f[:, index_c], marker="*")
f = flat_after_female_features; plt.scatter(x=f[:, 6], y=f[:, 7], c=f[:, index_c], marker="^")
f = flat_before_male_features; plt.scatter(x=f[:, 8], y=f[:, 9], c=f[:, index_c], marker="v");

In [ ]:
puzzle = {}  # key=(i, j), value=(piece, dx, dy, alpha)

# take a random corner piece
piece = [piece for piece in pieces if piece.nb_flats == 2][0]
angle_degrees = sum([edge.angle_degrees for edge in piece.edges if edge.type == 0])/2 + 45
corner_point = [edge.p1 for edge in piece.edges if edge.type == 0 and edge.next.type == 0][0]
h, w = piece.size
#dx = corner_point[0] - x
#dy = corner_point[1] - y
print(corner_point)
dx = 0
dy = 0
print(dx, dy)
puzzle[(0, 0)] = (piece, dx, dy, angle_degrees)
def puzzle_show(puzzle):
    img = np.zeros_like(img_gray)
    hh, ww = img.shape[:2]
    for piece, dx, dy, angle_degrees in puzzle.values():
        #matrix = cv2.getRotationMatrix2D((0, 0), 0, 1)
        #cv2.warpAffine(piece.img, matrix, (w, h), img)
        h, w = piece.size
        matrix = cv2.getRotationMatrix2D((w//2, h//2), angle_degrees, 1)
        cv2.warpAffine(piece.img, matrix, (ww, hh), img, cv2.INTER_LINEAR, cv2.BORDER_TRANSPARENT)
    plt.imshow(img) # [0:500, 0:500])
puzzle_show(puzzle)